Load data

In [93]:
import pandas as pd

# Load datasets
index_of_industrial_production = pd.read_excel('./Index of Industrial Production.xlsx')
npa_data = pd.read_excel('./Other STRBI Table No 20. Bank-Wise and Bank Group-Wise Gross Non-Performing Assets, Gross Advances, and Gross NPA Ratio of Scheduled Commercial Banks.xlsx')
forex_market_turnover = pd.read_excel('./Turnover in Foreign Exchange Market - Daily.xlsx')

Filter data (focusing on BOB)

In [94]:
# Filter the NPA dataset for BOB
bob_npa_data = npa_data[npa_data['Unnamed: 1'] == 'BANK OF BARODA']

In [105]:
bob_npa_data['Unnamed: 0'] = pd.to_datetime(bob_npa_data['Unnamed: 0'], format='%Y')

C:\Users\Somya\AppData\Local\Temp\ipykernel_21752\211381859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bob_npa_data['Unnamed: 0'] = pd.to_datetime(bob_npa_data['Unnamed: 0'], format='%Y')


In [106]:
bob_npa_data.head()

,Unnamed: 0,Unnamed: 1,"As on March 31, 2023",Unnamed: 3,Unnamed: 4
1,2023-01-01,BANK OF BARODA,36763.69,969548.31,3.791837
90,2022-01-01,BANK OF BARODA,54059.39,818120.5,6.607754
179,2021-01-01,BANK OF BARODA,66670.99,751590.16,8.870658
270,2020-01-01,BANK OF BARODA,69381.43,738096.45,9.400049
366,2019-01-01,BANK OF BARODA,48232.77,501706.39,9.613744


In [95]:
industrial_production_relevant = index_of_industrial_production

In [96]:
# Extract turnover relevant for BOB
forex_market_columns = ['Date', 'Purchases', 'Sales']
forex_market_relevant = forex_market_turnover[forex_market_columns]

Merging & preprocessing

In [97]:
# Preprocess Date col forex_market_relevant 
forex_market_relevant['Date'] = pd.to_datetime(forex_market_relevant['Date'], errors='coerce')
forex_market_relevant = forex_market_relevant.dropna(subset=['Date'])

C:\Users\Somya\AppData\Local\Temp\ipykernel_21752\2399358424.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forex_market_relevant['Date'] = pd.to_datetime(forex_market_relevant['Date'], errors='coerce')


In [98]:
forex_market_relevant = forex_market_relevant.sort_values('Date')

In [99]:
forex_market_relevant.head()

,Date,Purchases,Sales
6416,1998-08-03,210,216
6415,1998-08-04,193,210
6414,1998-08-05,182,236
6413,1998-08-06,230,187
6412,1998-08-07,194,165


In [100]:
industrial_production_relevant = industrial_production_relevant.T
industrial_production_relevant.columns = industrial_production_relevant.iloc[0]
industrial_production_relevant = industrial_production_relevant.drop("Item Description")
industrial_production_relevant.index = pd.to_datetime(industrial_production_relevant.index, errors='coerce', format='%Y:%m (%b)')

In [101]:
industrial_production_relevant = industrial_production_relevant.sort_index()

In [102]:
industrial_production_relevant.head()

Item Description,General Index,Manufacture of basic metals,Manufacture of beverages,Manufacture of chemicals and chemical products,Manufacture of coke and refined petroleum products,"Manufacture of computer, electronic and optical products",Manufacture of electrical equipment,"Manufacture of fabricated metal products, except machinery and equipment",Manufacture of food products,Manufacture of furniture,...,"Manufacture of pharmaceuticals, medicinal chemical and botanical products",Manufacture of rubber and plastics products,Manufacture of textiles,Manufacture of tobacco products,Manufacture of wearing apparel,Other manufacturing,Printing and reproduction of recorded media,NaN,NaN,Notes : Data are provisional for latest three months.
2012-04-01,99.3,104.1,134.2,100.4,96.0,92.0,93.8,92.2,97.3,98.0,...,85.2,94.7,98.0,105.0,94.5,108.5,87.5,NaN,NaN,NaN
2012-05-01,105.0,119.3,147.1,102.9,103.4,95.4,108.1,102.8,91.7,107.4,...,96.4,99.7,106.5,105.6,93.1,116.5,99.4,NaN,NaN,NaN
2012-06-01,102.3,106.4,130.5,102.6,103.7,102.8,110.3,101.9,86.1,112.5,...,120.2,101.3,103.4,96.3,89.5,139.4,101.9,NaN,NaN,NaN
2012-07-01,101.5,109.5,93.1,104.9,105.7,107.2,109.8,86.8,88.1,105.1,...,119.4,103.3,108.7,96.5,87.4,116.0,94.6,NaN,NaN,NaN
2012-08-01,99.9,101.3,85.1,106.5,105.7,102.5,125.8,86.5,85.9,110.7,...,123.7,99.3,113.1,86.2,90.1,121.1,90.1,NaN,NaN,NaN


Merge data

In [107]:
# Merge Forex Market Data with Industrial Production Index
merged_data = pd.merge_asof(forex_market_relevant.sort_values('Date'), 
                            industrial_production_relevant, 
                            left_on='Date', right_index=True, 
                            direction='nearest')

In [108]:
merged_data['Date'] = pd.to_datetime(merged_data['Date'])

In [109]:
merged_data = pd.merge_asof(merged_data.sort_values('Date'), 
                            bob_npa_data.sort_values('Unnamed: 0'), 
                            left_on='Date', right_on='Unnamed: 0', 
                            direction='nearest')

In [110]:
merged_data.head()

,Date,Purchases,Sales,General Index,Manufacture of basic metals,Manufacture of beverages,Manufacture of chemicals and chemical products,Manufacture of coke and refined petroleum products,"Manufacture of computer, electronic and optical products",Manufacture of electrical equipment,...,Other manufacturing,Printing and reproduction of recorded media,NaN,NaN,Notes : Data are provisional for latest three months.,Unnamed: 0,Unnamed: 1,"As on March 31, 2023",Unnamed: 3,Unnamed: 4
0,1998-08-03,210,216,99.3,104.1,134.2,100.4,96.0,92.0,93.8,...,108.5,87.5,NaN,NaN,NaN,2004-01-01,BANK OF BARODA,3798.95,31238.81,12.160995
1,1998-08-04,193,210,99.3,104.1,134.2,100.4,96.0,92.0,93.8,...,108.5,87.5,NaN,NaN,NaN,2004-01-01,BANK OF BARODA,3798.95,31238.81,12.160995
2,1998-08-05,182,236,99.3,104.1,134.2,100.4,96.0,92.0,93.8,...,108.5,87.5,NaN,NaN,NaN,2004-01-01,BANK OF BARODA,3798.95,31238.81,12.160995
3,1998-08-06,230,187,99.3,104.1,134.2,100.4,96.0,92.0,93.8,...,108.5,87.5,NaN,NaN,NaN,2004-01-01,BANK OF BARODA,3798.95,31238.81,12.160995
4,1998-08-07,194,165,99.3,104.1,134.2,100.4,96.0,92.0,93.8,...,108.5,87.5,NaN,NaN,NaN,2004-01-01,BANK OF BARODA,3798.95,31238.81,12.160995


Feature Engineering

In [115]:
merged_data['year'] = merged_data['Date'].dt.year
merged_data['month'] = merged_data['Date'].dt.month

In [116]:
merged_data['NPA_to_Advances_Ratio'] = merged_data['As on March 31, 2023'] / merged_data['Unnamed: 3']

In [120]:
merged_data['NPA_to_Advances_Ratio_Lag1'] = merged_data['NPA_to_Advances_Ratio'].shift(1)
merged_data['NPA_to_Advances_Ratio_Lag3'] = merged_data['NPA_to_Advances_Ratio'].shift(3)
merged_data['NPA_to_Advances_Ratio_RollMean'] = merged_data['NPA_to_Advances_Ratio'].rolling(window=3).mean()
merged_data['NPA_to_Advances_Ratio_RollStd'] = merged_data['NPA_to_Advances_Ratio'].rolling(window=3).std()

In [123]:
# threshold for operational risk
threshold = merged_data['NPA_to_Advances_Ratio'].quantile(0.90)

# Create the target variable
merged_data['Operational_Risk'] = (merged_data['NPA_to_Advances_Ratio'] > threshold).astype(int)

In [124]:
merged_data.fillna(method='ffill', inplace=True)

C:\Users\Somya\AppData\Local\Temp\ipykernel_21752\2235490398.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='ffill', inplace=True)


In [125]:
features = merged_data[['year', 'month', 'NPA_to_Advances_Ratio', 'NPA_to_Advances_Ratio_Lag1', 'NPA_to_Advances_Ratio_Lag3', 'NPA_to_Advances_Ratio_RollMean', 'NPA_to_Advances_Ratio_RollStd']]
target = merged_data['Operational_Risk']

Model

In [137]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

features_filled = features.fillna(method='ffill').fillna(method='bfill')

features_reduced = features_filled[['year', 'month', 'NPA_to_Advances_Ratio']]

X_train, X_test, y_train, y_test = train_test_split(features_reduced, target, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

classifier = LogisticRegression(random_state=42)
classifier.fit(X_train_balanced, y_train_balanced)

cv_scores = cross_val_score(classifier, X_train_balanced, y_train_balanced, cv=5)
print("Cross-validation scores:", cv_scores)
print("Average cross-validation score:", cv_scores.mean())

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test data:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

C:\Users\Somya\AppData\Local\Temp\ipykernel_21752\2852053579.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_filled = features.fillna(method='ffill').fillna(method='bfill')


Cross-validation scores: [0.8958544  0.88523761 0.87411527 0.8983308  0.87000506]
Average cross-validation score: 0.8847086284997389
Accuracy on test data: 0.7895557287607171
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.78      0.88      1239
           1       0.14      1.00      0.25        44

    accuracy                           0.79      1283
   macro avg       0.57      0.89      0.56      1283
weighted avg       0.97      0.79      0.86      1283

